In [ ]:
import time, hashlib, requests, json

In [ ]:
# Marvel API data
endpoint = "https://gateway.marvel.com/"
getCharsMethod = "v1/public/comics"
apiKey = "6818695382c18243570eb27db6d83eb3"
privateKey = "0ca11a0a359ac649d46d934064349266dde56d3d"
limit = "100"

In [ ]:
# Open file for storing the database
comicsDb = open('data/marvel_db_comics.json', 'w')

# Clear the file to update it
comicsDb.seek(0)
comicsDb.truncate()

# Total number of comics variable
totalComicsFetched = 0
totalComics = 0

In [ ]:
comics = []
firstIterationFlag = True
# Cycle through the database
for offset in range(0, 40900, 100):
    
    # Generate md5 hash from both api keys and timestamp
    timestamp = str(int(time.time()))
    hs = hashlib.md5(timestamp.encode('utf-8') + privateKey.encode('utf-8') + apiKey.encode('utf-8')).hexdigest()
    
    # Make the request
    response = requests.get(endpoint + getCharsMethod
                            + "?" + "orderBy=title"
                            + "&" + "offset=" + str(offset) 
                            + "&" + "limit=" + limit
                            + "&" + "ts=" + timestamp 
                            + "&" + "apikey=" + apiKey
                            + "&" + "hash=" + hs)
    
    # Break if at least 1 
    # request was unsuccessful
    if(response.status_code != 200):
        print("An error " + str(response.status_code) + " occured!" + str(response.json()))
        break
    
    # Extract the data about characters
    data = response.json()['data']
    
    # Increment total characters fetched variable
    totalComicsFetched += data['count']
    # Get the total number of characters available via API
    if(firstIterationFlag):
        totalComics = data['total']
        firstIterationFlag = False
    
    # Write results to the file
    results = data['results']
    for comic in results:
        comics.append(comic)
    
    # Sleep for some time
    # time.sleep(1)

# Write to the file
json.dump(comics, comicsDb, indent=2)

In [ ]:
# Check if all characters are fetched
print('Total comics available via Marvel API: ' + str(totalComics))
print('Comics fetched: ' + str(totalComicsFetched))
if(totalComicsFetched == totalComics):
    print('OK! All comics has been fetched!')
else:
    print('Error! Only ' + str(totalComicsFetched) + '/' + str(totalComics) + ' have been fetched!' )
# Close database file
comicsDb.close()